In [6]:
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('Fashion Dataset.csv').head(1000)

In [8]:
data.head(5)

,Unnamed: 0,p_id,name,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,0,17048614.0,Khushal K Women Black Ethnic Motifs Printed Ku...,5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,1,16524740.0,InWeave Women Orange Solid Kurta with Palazzos...,5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,2,16331376.0,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,3,14709966.0,Nayo Women Red Floral Printed Kurta With Trous...,3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,4,11056154.0,AHIKA Women Black & Green Printed Straight Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [9]:
data.columns

Index(['Unnamed: 0', 'p_id', 'name', 'price', 'colour', 'brand', 'img',
       'ratingCount', 'avg_rating', 'description', 'p_attributes'],
      dtype='object')

In [10]:
data.shape

(1000, 11)

In [11]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

for index, row in data.iterrows():
    # Check if 'name' and 'description' are floats (indicating NaN values) and handle them
    if isinstance(row['name'], float):
        tokens_title = []
    else:
        # Tokenize and remove stopwords for title
        tokens_title = [word.lower() for word in word_tokenize(str(row['name'])) if word.isalpha() and word.lower() not in stop_words]
    
    if isinstance(row['description'], float):
        tokens_description = []
    else:
        # Tokenize and remove stopwords for description
        tokens_description = [word.lower() for word in word_tokenize(row['description']) if word.isalpha() and word.lower() not in stop_words]
    
    # Print tokenized text for each row
    print(f"Row {index}:")
    print("Title tokens:", tokens_title)
    print("Description tokens:", tokens_description)
    print("\n")


Row 0:
Title tokens: ['khushal', 'k', 'women', 'black', 'ethnic', 'motifs', 'printed', 'kurta', 'palazzos', 'dupatta']
Description tokens: ['black', 'printed', 'kurta', 'palazzos', 'dupatta', 'br', 'br', 'b', 'kurta', 'design', 'ul', 'li', 'ethnic', 'motifs', 'printed', 'li', 'anarkali', 'shape', 'li', 'regular', 'style', 'li', 'mandarin', 'collar', 'regular', 'sleeves', 'li', 'calf', 'length', 'flared', 'hem', 'li', 'viscose', 'rayon', 'machine', 'weave', 'fabric', 'br', 'b', 'palazzos', 'design', 'ul', 'li', 'printed', 'palazzos', 'li', 'elasticated', 'waistband', 'li', 'closure', 'dupatta', 'length', 'meters', 'width', 'nbsp', 'cm', 'br', 'model', 'height', 'wearing', 'size', 'rayon', 'br', 'machine', 'wash']


Row 1:
Title tokens: ['inweave', 'women', 'orange', 'solid', 'kurta', 'palazzos', 'floral', 'print', 'dupatta']
Description tokens: ['orange', 'solid', 'kurta', 'palazzos', 'dupatta', 'br', 'br', 'b', 'kurta', 'design', 'ul', 'li', 'solid', 'li', 'shape', 'li', 'regular', 'st

In [12]:
import torch
from transformers import BertTokenizer, BertModel

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define a function to get BERT embeddings
def get_bert_embeddings(text):
    if not isinstance(text, str):
        text = str(text)
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]

# Get BERT embeddings for 'name' and 'description' columns
bert_name = data['name'].apply(get_bert_embeddings)
bert_description = data['description'].apply(get_bert_embeddings)

# Print the BERT embeddings
print("BERT Name embeddings shape:", bert_name.shape)
print("BERT Description embeddings shape:", bert_description.shape)


BERT Name embeddings shape: (1000,)
BERT Description embeddings shape: (1000,)


In [13]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Load pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to get BERT embeddings for a text
def get_bert_embeddings(text):
    # Ensure text is a string
    text = str(text)
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract the embeddings corresponding to the [CLS] token (first token)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Create an empty DataFrame to store textual features
textual_features = pd.DataFrame(columns=['name_embedding', 'description_embedding'])

# Assuming 'data' is your DataFrame containing the dataset
for index, row in data.iterrows():
    # Get BERT embeddings for name and description
    name_embedding = get_bert_embeddings(row['name'])
    description_embedding = get_bert_embeddings(row['description'])
    
    # Concatenate or average the embeddings
    concatenated_embedding = np.concatenate((name_embedding, description_embedding))
    averaged_embedding = np.mean([name_embedding, description_embedding], axis=0)
    
    # Append the embeddings to the textual_features DataFrame
    textual_features = pd.concat([textual_features, pd.DataFrame({'name_embedding': [name_embedding],
                                                                  'description_embedding': [description_embedding]})],
                                  ignore_index=True)

# Print the textual_features DataFrame
print(textual_features)


                                        name_embedding  \
0    [-0.7064648, -0.15478453, 0.01975272, -0.12516...   
1    [-0.69479334, -0.22591954, 0.4031542, -0.33173...   
2    [-0.5612235, -0.38948515, -0.076061994, -0.145...   
3    [-0.4503839, -0.49103975, -0.2914814, -0.04026...   
4    [-0.36487466, -0.3705552, -0.2932217, -0.02928...   
..                                                 ...   
995  [-0.43305287, -0.8569128, -0.549504, 0.0538934...   
996  [-0.258075, -0.23850498, -0.1844844, 0.2749457...   
997  [-0.33041608, -0.3829472, -0.36472583, 0.07243...   
998  [-0.2154162, -0.02203576, 0.1181871, 0.0865691...   
999  [-0.39003915, -0.15357089, -0.07198691, -0.115...   

                                 description_embedding  
0    [-0.8761024, -0.26375538, -0.09928207, -0.2241...  
1    [-0.53199726, -0.45348433, -0.12778379, -0.268...  
2    [-0.6729017, -0.32411295, -0.40147394, -0.1657...  
3    [-0.6857533, -0.52290875, -0.20446342, -0.2515...  
4    [-0.52014565,

In [14]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os
import numpy as np

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

model = models.resnet18(pretrained=True)
model.eval()

folder_path = "Images/Images"
image_paths = os.listdir(folder_path)[:1000]  

# Function to extract visual features
def extract_features(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  
    with torch.no_grad():
        features = model(image)
    return features.numpy().flatten()  # Flatten the features to store as 1D array

# Extract visual features for each image and store them
visual_features = []
for image_name in image_paths:
    image_path = os.path.join(folder_path, image_name)
    features = extract_features(image_path)
    visual_features.append(features)

# Convert visual_features list to numpy array
visual_features = np.array(visual_features)

# Save visual features to a file
np.save("visual_features.npy", visual_features)



C:\Users\Guhan\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Guhan\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
import pandas as pd

# Assuming 'visual_features.npy' contains the visual features saved from the previous code
visual_features = np.load("visual_features.npy")

# Create a DataFrame named visual_features_df
visual_features_df = pd.DataFrame(visual_features)

# Display the visual features DataFrame
print(visual_features_df)


          0         1         2         3         4         5         6    \
0    0.320139 -0.615698 -2.485633 -2.011794 -2.348554 -0.054532  2.822530   
1   -2.389232 -3.249185 -0.746811 -3.953593 -2.166986 -3.457104 -1.159570   
2   -2.327149 -4.012034 -2.065993 -0.770661  0.905337 -0.400016  2.101400   
3   -0.228192 -2.536293 -0.601009 -2.277702  0.953265 -0.882404 -0.433705   
4   -0.658074  2.709488 -3.709250 -3.427626 -5.276134 -4.361906 -2.696824   
..        ...       ...       ...       ...       ...       ...       ...   
995 -0.791206 -1.569288 -2.306022 -1.879139 -0.105875 -1.112896 -0.017538   
996 -0.254125 -1.783991 -1.845293 -2.064206  0.610216 -0.198207  1.173456   
997  1.364189 -2.097882 -2.269563 -1.756323 -1.583330 -0.840086  1.756295   
998 -1.581471  0.346274 -1.089204 -0.387645 -1.212886 -2.344428  0.489235   
999  0.184221 -3.130453 -1.213871 -2.300966 -1.923739 -2.048238 -2.844033   

          7         8         9    ...       990       991       992  \
0  

In [16]:

# Display the size of the visual_features_df
print("Visual features DataFrame size:", visual_features_df.shape)

# Display the size of the textual_features DataFrame
print("Textual features DataFrame size:", textual_features.shape)


Visual features DataFrame size: (1000, 1000)
Textual features DataFrame size: (1000, 2)


In [17]:

# Concatenate textual and visual features
multimodal_features = pd.concat([textual_features, visual_features_df], axis=1)
# Print the shape of the multimodal features DataFrame
print("Multimodal features shape:", multimodal_features.shape)


Multimodal features shape: (1000, 1002)


In [18]:
textual_feature_size=textual_features.shape
visual_feature_size=visual_features_df.shape

In [20]:
import tensorflow as tf
from tensorflow.keras import layers

def create_multimodal_recommendation_model(text_input_shape, image_input_shape):
    # Text input branch
    text_input = layers.Input(shape=text_input_shape, name='text_input')
    text_dense = layers.Dense(128, activation='relu')(text_input)

    # Image input branch
    image_input = layers.Input(shape=image_input_shape, name='image_input')
    image_dense = layers.Dense(128, activation='relu')(image_input)

    # Concatenate textual and visual features
    concatenated = layers.Concatenate()([text_dense, image_dense])

    # Recommendation output
    output = layers.Dense(1, activation='sigmoid')(concatenated)  

    # Create model
    model = tf.keras.Model(inputs=[text_input, image_input], outputs=output)

    return model

text_input_shape = (2,)  # Shape of textual features
image_input_shape = (1000,)  # Shape of visual features
model = create_multimodal_recommendation_model(text_input_shape, image_input_shape)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_input         │ (None, 1000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │        384 │ text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │    128,128 │ image_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        257 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 128,769 (503.00 KB)

 Trainable params: 128,769 (503.00 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
X_train_text = np.random.rand(1000, 2)
X_train_image = np.random.rand(1000, 1000)
y_train = np.random.randint(0, 2, size=(1000,))

# Placeholder for your test data (replace with your actual data)
X_test_text = np.random.rand(200, 2)
X_test_image = np.random.rand(200, 1000)
y_test = np.random.randint(0, 2, size=(200,))

# Now you can train the model using these variables
history = model.fit(x=[X_train_text, X_train_image], y=y_train, batch_size=32, epochs=10, validation_split=0.2)

# Evaluate the model on the test set after training
test_loss, test_accuracy = model.evaluate(x=[X_test_text, X_test_image], y=y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4935 - loss: 0.7500 - val_accuracy: 0.5750 - val_loss: 0.7140
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5205 - loss: 0.7164 - val_accuracy: 0.5150 - val_loss: 0.7477
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5734 - loss: 0.6805 - val_accuracy: 0.5450 - val_loss: 0.7358
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5873 - loss: 0.6729 - val_accuracy: 0.5250 - val_loss: 0.6860
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6469 - loss: 0.6390 - val_accuracy: 0.5600 - val_loss: 0.6909
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7087 - loss: 0.5873 - val_accuracy: 0.5600 - val_loss: 0.6956
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7136 - loss: 0.5770 - val_accuracy: 0.5400 - val_loss: 0.7269
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7038 - loss: 0.5644 - val_accuracy: 0.5400 - val_loss:

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import average_precision_score, ndcg_score
from sklearn.model_selection import train_test_split

X_train_text, X_val_text, X_train_image, X_val_image, y_train, y_val = train_test_split(
    X_train_text, X_train_image, y_train, test_size=0.2, random_state=42)


y_pred_val = model.predict([X_val_text, X_val_image])

y_pred_binary_val = (y_pred_val > 0.5).astype(int)

# Compute classification metrics
accuracy_val = accuracy_score(y_val, y_pred_binary_val)
precision_val = precision_score(y_val, y_pred_binary_val)
recall_val = recall_score(y_val, y_pred_binary_val)
f1_score_val = f1_score(y_val, y_pred_binary_val)

average_precision_val = average_precision_score(y_val, y_pred_val.flatten())  # Flatten y_pred_val
ndcg_val = ndcg_score([y_val], [y_pred_val.flatten()])  # Flatten both y_val and y_pred_val

# Print the evaluation metrics
print("Validation Accuracy:", accuracy_val)
print("Validation Precision:", precision_val)
print("Validation Recall:", recall_val)
print("Validation F1-score:", f1_score_val)
print("Validation Average Precision (AP):", average_precision_val)
print("Validation NDCG:", ndcg_val)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Validation Accuracy: 0.83125
Validation Precision: 0.7816091954022989
Validation Recall: 0.8947368421052632
Validation F1-score: 0.8343558282208589
Validation Average Precision (AP): 0.922174451259384
Validation NDCG: 0.9853454943293264


In [44]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the model architecture
def create_multimodal_recommendation_model(text_input_shape, image_input_shape):
    # Text input branch
    text_input = layers.Input(shape=text_input_shape, name='text_input')
    text_dense = layers.Dense(128, activation='relu')(text_input)

    # Image input branch
    image_input = layers.Input(shape=image_input_shape, name='image_input')
    image_dense = layers.Dense(128, activation='relu')(image_input)

    # Concatenate textual and visual features
    concatenated = layers.Concatenate()([text_dense, image_dense])

    # Recommendation output
    output = layers.Dense(1, activation='sigmoid')(concatenated)  # Output 1 neuron for binary classification

    # Create model
    model = tf.keras.Model(inputs=[text_input, image_input], outputs=output)

    return model

# Example usage:
text_input_shape = (2,)  # Shape of textual features
image_input_shape = (1000,)  # Shape of visual features
model = create_multimodal_recommendation_model(text_input_shape, image_input_shape)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Now you can train, evaluate, and make predictions using your data.
history = model.fit(
    x=[X_train_text, X_train_image],
    y=y_train,
    batch_size=32,
    epochs=10,
    validation_data=([X_val_text, X_val_image], y_val)
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate([X_test_text, X_test_image], y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_input         │ (None, 1000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │        384 │ text_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │    128,128 │ image_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 256)       │          0 │ dense_9[0][0],    │
│ (Concatenate)       │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │        257 │ concatenate_3[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 128,769 (503.00 KB)

 Trainable params: 128,769 (503.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.4611 - loss: 0.8734 - val_accuracy: 0.4750 - val_loss: 0.7316
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5283 - loss: 0.7052 - val_accuracy: 0.4938 - val_loss: 0.7008
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5360 - loss: 0.6895 - val_accuracy: 0.5250 - val_loss: 0.6948
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6476 - loss: 0.6589 - val_accuracy: 0.4812 - val_loss: 0.7135
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5608 - loss: 0.6629 - val_accuracy: 0.5188 - val_loss: 0.6840
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6160 - loss: 0.6563 - val_accuracy: 0.5375 - val_loss: 0.6836
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6360 - loss: 0.6344 - val_accuracy: 0.4812 - val_loss: 0.7307
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6139 - loss: 0.6141 - val_accuracy: 0.5500 - val_

In [42]:
import numpy as np

# Take one random sample from the test data
random_index = np.random.randint(0, len(X_test_text))
sample_text_features = X_test_text[random_index]
sample_image_features = X_test_image[random_index]

# Reshape the features for model input
sample_text_features = sample_text_features.reshape(1, -1)
sample_image_features = sample_image_features.reshape(1, -1)

# Make prediction using the model
prediction = model.predict([sample_text_features, sample_image_features])

# Print the prediction
print("Prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Prediction: [[0.6110271]]
